In [ ]:
import uuid
from typing import Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START
from langgraph.graph import StateGraph
from langgraph.types import interrupt
from langgraph.types import Command
class State(TypedDict):
    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""

def node (state: State):
    answer = interrupt(
        "what is your  age?"
    )
    print(f"> Received an input from the interrupt : {answer}")
    return{ "human_value": answer}

builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")

#A checkpointer must be enabled for interrupts to work!
checkpointer = MemorySaver()

graph = builder.compile(checkpointer= checkpointer)

config= {
    "configurable" : {
        "thread_id": uuid.uuid4(),
    }
}

for chunk in graph.stream({"food": "abc"}, config):
    print(chunk)


In [ ]:
command = Command(resume="some input from users")
for  chunk in graph.stream(Command(resume="Some input from users"), config):
    print(chunk)